In [1]:
import pandas as pd
from scipy.stats import norm
from numpy import sqrt

table_a = 'Amazon'
table_b = 'BarnesAndNoble'
candidate_set = 'Falcon1_apply_rules_ds'
prediction_set = 'Falcon1_cm_matching_al_ds'
dfa = pd.read_csv(table_a)
dfb = pd.read_csv(table_b)
dfc = pd.read_csv(candidate_set)
dfp = pd.read_csv(prediction_set)

shuffled_dfc = dfc.sample(frac=1)
# shuffled_dfc.to_csv('shuffled_dfc', index=False)

In [2]:
import re
at = 'American War: A Novel'
bt = 'American War'
atitle = re.sub('[^0-9a-zA-Z]+', ' ', at).split(' ')
btitle = re.sub('[^0-9a-zA-Z]+', ' ', bt).split(' ')
common_words = list(set(atitle) & set(btitle))
total_words = list(set(atitle) | set(btitle))
fraction = len(common_words) / len(total_words)
fraction

0.5

In [3]:
import re
def blocking_rules():
    new_cs = {}
    j = 0
    for i in range(len(shuffled_dfc)):
        tuple = shuffled_dfc.iloc[i, :]
        idxa = tuple.A_id; idxb = tuple.B_id
        rowa = dfa.iloc[idxa, :]
        rowb = dfb.iloc[idxb, :]
        # Blocking Rule 1: Block all with missing `title` value
        if type(rowa['title']) == str and type(rowb['title']) == str and len(rowa['title']) > 0 and len(rowb['title']) > 0:
            # Blocking Rule 2: Block tuple pairs with non-matching `book_format`
            if rowa['book_format'] == rowb['book_format']:
                # Blocking Rule 3: Fraction of common words in title of tuple pairs > threshold(0.25)
                atitle = re.sub('[^0-9a-zA-Z]+', ' ', rowa['title']).split(' ')
                btitle = re.sub('[^0-9a-zA-Z]+', ' ', rowb['title']).split(' ')
                common_words = list(set(atitle) & set(btitle))
                total_words = list(set(atitle) | set(btitle))
                fraction = len(common_words) / len(total_words)
                if fraction > 0.10:
                    new_cs[j] = tuple
                    j = j+1
        
    updated_cdf = pd.DataFrame.from_dict(new_cs)
    updated_cdf = updated_cdf.T
    updated_cdf.to_csv('updated_cdf_afterblocking3.csv', index=False)
    
    return len(new_cs)
blocking_rules()

1520

In [4]:
shuffled_dfc = pd.read_csv('updated_cdf_afterblocking3.csv')
print('Len of new shuffled_dfc', len(shuffled_dfc))

Len of new shuffled_dfc 1520


In [7]:

def get_sample(start=0, n=50):
    count_pos = 0
    current_sample = {}
    k = start
    cols = dfa.columns.tolist()
    cols.remove('Up_System')
    cols.remove('_id')
    cols = ['_id', 'Up_System', 'predicted', 'manual'] + cols
    for i in range(len(shuffled_dfc[start:start+n])):
        print(start + i)
        tuple = shuffled_dfc.iloc[start + i, :]
        idxa = tuple.A_id; idxb = tuple.B_id
        rowa = dfa.iloc[idxa, :]
        rowb = dfb.iloc[idxb, :]
        rowa['Up_System'] = str(start + i) + 'A'
        rowb['Up_System'] = str(start + i) + 'B'
        rowa['manual'] = ''
        rowb['manual'] = ''
        matching = dfp[(dfp['id1'] == rowa['_id']) & (dfp['id2'] == rowb['_id'])]
        if len(matching) == 1 and matching['id1'].values[0] == rowa['_id'] and matching['id2'].values[0] == rowb['_id']:
            count_pos += 1
            rowa['predicted'] = True
            rowb['predicted'] = True
        else:
            rowa['predicted'] = False
            rowb['predicted'] = False
        #print(pd.Series.to_dict(rowa))
        current_sample[k] = pd.Series.to_dict(rowa)
        current_sample[k+1] = pd.Series.to_dict(rowb)
        k += 2

    Sdf = pd.DataFrame.from_dict(current_sample)
    Sdf = Sdf.T
    #Sdf.to_csv('run'+str(start) + '_' + str(start+n) + '.csv', index=False, columns=cols)
    Sdf.to_csv('all1520.csv', index=False, columns=cols)
    return count_pos
get_sample(0, 2000)

0
ID1 1199
ID2 3713
1


/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-d

2
ID1 2567
ID2 5280
3
4
ID1 1148
ID2 4488
5
6
7
8
ID1 3227
ID2 745
9
10
11
12
13
ID1 2865
ID2 5304
14
15
16
17
ID1 1593
ID2 2876
18
ID1 207
ID2 2459
19
ID1 2043
ID2 2255
20
21
ID1 1136
ID2 5023
22
ID1 4050
ID2 1462
23
24
ID1 1701
ID2 5344
25
ID1 1474
ID2 1567
26
27
28
ID1 1793
ID2 4166
29
30
31
ID1 786
ID2 1423
32
ID1 3667
ID2 5300
33
34
35
ID1 2051
ID2 4642
36
37
38
ID1 3969
ID2 4967
39
ID1 2005
ID2 3720
40
41
ID1 2750
ID2 3034
42
43
ID1 1616
ID2 3796
44
ID1 1938
ID2 1157
45
ID1 3065
ID2 2179
46
ID1 686
ID2 2367
47
ID1 4064
ID2 2602
48
49
50
51
ID1 1084
ID2 4303
52
ID1 2630
ID2 2523
53
54
ID1 2099
ID2 3665
55
56
ID1 2040
ID2 2038
57
ID1 2253
ID2 4376
58
59
60
61
62
ID1 1177
ID2 5123
63
64
65
66
67
ID1 1006
ID2 2837
68
69
ID1 3944
ID2 701
70
71
72
ID1 3844
ID2 1673
73
ID1 2314
ID2 2425
74
75
76
ID1 2089
ID2 593
77
ID1 1733
ID2 4037
78
79
80
81
82
83
ID1 734
ID2 1849
84
85
ID1 362
ID2 613
86
ID1 1501
ID2 3547
87
88
89
ID1 1742
ID2 659
90
91
92
93
94
95
ID1 2205
ID2 4590
96
97
98
99
ID1 

744
ID1 2207
ID2 4903
745
ID1 935
ID2 4092
746
747
ID1 2062
ID2 136
748
749
ID1 223
ID2 5269
750
751
752
753
ID1 1038
ID2 4678
754
ID1 3117
ID2 5302
755
756
757
758
759
ID1 1978
ID2 2702
760
ID1 1746
ID2 2864
761
ID1 3593
ID2 1555
762
763
764
ID1 1832
ID2 2474
765
ID1 3903
ID2 1779
766
767
768
ID1 3295
ID2 5099
769
ID1 2207
ID2 4903
770
ID1 3815
ID2 487
771
772
773
774
ID1 1028
ID2 4957
775
ID1 2605
ID2 1439
776
ID1 626
ID2 1285
777
ID1 1406
ID2 3408
778
ID1 4154
ID2 4255
779
780
ID1 3880
ID2 1327
781
782
783
784
785
786
787
ID1 1668
ID2 4221
788
789
790
ID1 3716
ID2 1851
791
792
793
794
ID1 3936
ID2 4876
795
ID1 1036
ID2 3501
796
797
798
ID1 685
ID2 495
799
800
801
802
803
ID1 1704
ID2 1194
804
805
ID1 3601
ID2 142
806
ID1 2112
ID2 5419
807
ID1 1850
ID2 3434
808
ID1 2632
ID2 4658
809
810
811
ID1 2797
ID2 4944
812
813
ID1 1316
ID2 2589
814
815
ID1 478
ID2 2516
816
817
818
ID1 1672
ID2 31
819
ID1 3971
ID2 4381
820
821
822
ID1 1699
ID2 4600
823
ID1 297
ID2 2775
824
ID1 1299
ID2 4171
825


1432
ID1 3920
ID2 4627
1433
ID1 3002
ID2 548
1434
1435
ID1 4574
ID2 1380
1436
ID1 2227
ID2 1947
1437
1438
ID1 1037
ID2 4467
1439
ID1 3671
ID2 4853
1440
1441
1442
ID1 281
ID2 2181
1443
ID1 1680
ID2 3565
1444
ID1 1077
ID2 1000
1445
1446
1447
ID1 1898
ID2 1007
1448
ID1 3349
ID2 2456
1449
1450
ID1 1014
ID2 1805
1451
1452
1453
1454
1455
ID1 1990
ID2 4048
1456
ID1 2525
ID2 4732
1457
1458
1459
1460
1461
1462
1463
ID1 4028
ID2 269
1464
1465
ID1 241
ID2 1738
1466
ID1 2420
ID2 4896
1467
1468
ID1 2120
ID2 129
1469
1470
1471
ID1 1816
ID2 1823
1472
ID1 4032
ID2 2974
1473
ID1 1249
ID2 5072
1474
1475
1476
1477
1478
ID1 404
ID2 2264
1479
ID1 2494
ID2 2321
1480
1481
1482
1483
ID1 3924
ID2 3026
1484
1485
ID1 205
ID2 3371
1486
ID1 2
ID2 3266
1487
1488
ID1 758
ID2 1235
1489
1490
1491
ID1 1902
ID2 2610
1492
ID1 2163
ID2 515
1493
1494
1495
1496
1497
ID1 2288
ID2 5247
1498
ID1 3509
ID2 1134
1499
ID1 2107
ID2 711
1500
ID1 1195
ID2 2338
1501
ID1 213
ID2 3471
1502
1503
1504
ID1 2173
ID2 4518
1505
1506
1507
ID1 

628

In [14]:
ldf = pd.read_csv('labelled400.csv')
labelled_pairs = {}
counter = 0
for i in range(0, len(ldf), 2):
    t1 = ldf.iloc[i, :]['_id']
    t2 = ldf.iloc[i+1, :]['_id']
    label = ldf.iloc[i, :]['manual']
    if label == 1:
        label = True
    else:
        label = False
    labelled_pairs[counter] = {'id1': t1, 'id2': t2, 'label': label}
    counter += 1

lpdf = pd.DataFrame.from_dict(labelled_pairs)
lpdf = lpdf.T
lpdf.to_csv('labelled_pairs.csv', index=False)

,id1,id2,label
0,1199,3713,True
1,1375,4764,False
2,2567,5280,True
3,2205,3788,False
4,1148,4488,True
